In [ ]:
import requests
import json
import pandas as pd
import time
import os
from pathlib import Path

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по разметке вакансий. Если в тексте есть описание вакансии, извлеки следующие поля:
vacancy, company, city, country, experience, employment, schedule, salary_from, salary_to, currency, skills, responsibilities, requirements, experience level.

Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.

Если зарплата в тексте не указана — верни salary_from и salary_to как null.
Если в тексте нет описания вакансии, верни null.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст вакансии:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.2,
                    "max_tokens": 2500
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"]
            content = content.replace("```json", "").replace("```", "").strip()

            if content.lower() == "null":
                return None

            return json.loads(content)

        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса (попытка {attempt+1}):", e)
            time.sleep(2)
        except json.JSONDecodeError as e:
            print("Ошибка парсинга JSON:", e)
            print("Сырой ответ:", content)
            return None

    return None

def process_vacancies(input_file, output_file, partial_file='output_partial.csv'):
    df = pd.read_csv(input_file)
    df["uid"] = df["date"].astype(str) + "_" + df.index.astype(str)  # уникальный ID

    # Восстановление прогресса
    if os.path.exists(partial_file):
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial["uid"])
        print(f"Найден частичный файл. Уже обработано: {len(processed_ids)} вакансий.")
    else:
        df_partial = pd.DataFrame()
        processed_ids = set()

    for i, row in df.iterrows():
        uid = row["uid"]
        if uid in processed_ids:
            continue

        print(f"Обработка вакансии {i+1}/{len(df)} (UID: {uid})...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        # Приведение ответа к словарю
        if not isinstance(result, dict):
            if isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict):
                result = result[0]
            else:
                result = None

        if result is None:
            row_result = {
                "uid": uid,
                "vacancy": "", "company": "", "city": "", "country": "",
                "experience": "", "employment": "", "schedule": "",
                "salary_from": None, "salary_to": None, "currency": "",
                "skills": [], "responsibilities": [], "requirements": [], "experience level": []
            }
        else:
            row_result = {
                "uid": uid,
                "vacancy": result.get("vacancy", ""),
                "company": result.get("company", ""),
                "city": result.get("city", ""),
                "country": result.get("country", ""),
                "experience": result.get("experience", ""),
                "employment": result.get("employment", ""),
                "schedule": result.get("schedule", ""),
                "salary_from": result.get("salary_from"),
                "salary_to": result.get("salary_to"),
                "currency": result.get("currency", ""),
                "skills": result.get("skills", []),
                "responsibilities": result.get("responsibilities", []),
                "requirements": result.get("requirements", []),
                "experience level": result.get("experience level", [])
            }
        # Сохраняем результат построчно
        df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2.5)  # пауза между запросами

    # Финальное сохранение
    df_merged = pd.merge(df, df_partial, on="uid", how="left")
    df_merged.drop(columns=["uid"], inplace=True)
    df_merged.to_csv(output_file, index=False)
    print(f"Готово! Финальный результат сохранён в {output_file}")

In [21]:
# -------------------------------
# Обработка всех CSV из папки
# -------------------------------
INPUT_DIR = Path("temp")
OUTPUT_DIR = Path("temp")
OUTPUT_DIR.mkdir(exist_ok=True)

csv_files = sorted(INPUT_DIR.glob("*.csv"))

for csv_path in csv_files:
    file_name = csv_path.stem
    output_file = OUTPUT_DIR / f"{file_name}_labeled.csv"
    partial_file = OUTPUT_DIR / f"{file_name}_partial.csv"

    print(f"\n=== Обработка файла: {csv_path.name} ===")
    process_vacancies(
        input_file=str(csv_path),
        output_file=str(output_file),
        partial_file=str(partial_file)
    )


=== Обработка файла: parsed_vacancies_habr.csv ===
Обработка вакансии 1/2076 (UID: 2025-08-06T08:03:50+03:00_0)...
Обработка вакансии 2/2076 (UID: 2025-08-06T08:48:39+03:00_1)...
Обработка вакансии 3/2076 (UID: 2025-08-06T08:47:11+03:00_2)...
Обработка вакансии 4/2076 (UID: 2025-08-06T08:43:36+03:00_3)...
Обработка вакансии 5/2076 (UID: 2025-08-06T08:03:50+03:00_4)...
Обработка вакансии 6/2076 (UID: 2025-08-06T07:27:02+03:00_5)...
Обработка вакансии 7/2076 (UID: 2025-08-06T07:27:02+03:00_6)...
Обработка вакансии 8/2076 (UID: 2025-08-06T07:02:26+03:00_7)...
Обработка вакансии 9/2076 (UID: 2025-08-06T04:52:46+03:00_8)...
Обработка вакансии 10/2076 (UID: 2025-08-06T06:59:19+03:00_9)...
Обработка вакансии 11/2076 (UID: 2025-08-06T03:27:21+03:00_10)...
Обработка вакансии 12/2076 (UID: 2025-08-06T03:28:58+03:00_11)...
Обработка вакансии 13/2076 (UID: 2025-08-06T01:00:54+03:00_12)...
Обработка вакансии 14/2076 (UID: 2025-08-05T21:27:58+03:00_13)...
Обработка вакансии 15/2076 (UID: 2025-08-05

In [ ]:
api_url = "https://api.mistral.ai/v1/chat/completions"
model_name = "mistral-small-latest"
api_key = ''

In [2]:
HEADERS = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [3]:
vacancies = [
    "Руководитель отдела аналитики",
    "Руководитель группы разработки",
    "DevOps-инженер",
    "Дата-сайентист",
    "Гейм-дизайнер",
    "Системный аналитик",
    "Директор по информационным технологиям (CIO)",
    "Программист, разработчик",
    "Руководитель проектов",
    "Технический директор (СТО)",
    "Менеджер продукта",
    "BI-аналитик, аналитик данных",
    "Бизнес-аналитик",
    "Тестировщик",
    "Аналитик",
    "Арт-директор, креативный директор",
    "Продуктовый аналитик",
    "Системный инженер",
    "Сетевой инженер",
    "Технический писатель",
    "Дизайнер, художник",
    "Методолог",
    "Специалист по информационной безопасности",
    "Системный администратор",
    "Специалист технической поддержки",
    "Другая IT вакансия"
    "Не IT вакансия"
]

In [12]:
import requests
import json
import pandas as pd
import os
from pathlib import Path
import urllib3
import time
import re

# Отключаем предупреждения HTTPS
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# функция отправляет запрос по API и получает ответ
def send_request_to_vllm(api_url, prompt, model_name="default-model", api_key = None, max_tokens=8192, temperature=0.1, headers=HEADERS, top_p=0.6,
                         length_penalty=1.6, repetition_penalty=1.3, no_repeat_ngram_size=3, do_sample=False, top_k=100, stream=False, delay=2.5):
    """
    Отправляет POST-запрос в VLLM сервис для генерации текста.

    :param api_url: URL API сервиса VLLM
    :param prompt: Текст запроса (промпт) для модели
    :param model_name: Имя модели (опционально)
    :param max_tokens: Максимальное количество токенов в ответе (опционально)
    :param temperature: Параметр температуры для генерации (опционально)
    :param headers: Заголовки запроса (опционально)
    :return: Ответ от сервиса в формате JSON
    """
    time.sleep(delay)
    
    # Подготовка данных для отправки
    payload = {"model": model_name,
            "messages": [
                {"role": "system", "content": "Ты — ИИ-ассистент для обработки и структурирования навыков в IT-рекрутинге."},
                {"role": "user", "content": prompt}
            ],
            "temperature": temperature,
            #"top_p": top_p,
            #"max_new_tokens": max_tokens,
            #"length_penalty": length_penalty,
            #"early_stopping": True,
            #"repetition_penalty": repetition_penalty,
            #"no_repeat_ngram_size": no_repeat_ngram_size,
            #"do_sample": do_sample,
            #"top_k": top_k,
            #'stream': False  # Выключаем потоковую передачу
            "max_tokens": max_tokens
        }

    # Если заголовки не переданы, используем стандартные
    if headers is None:
        headers = {
            'Content-Type': 'application/json',
        }
    if api_key is not None:
        headers["Authorization"] = f"Bearer {api_key}"
    # Преобразуем данные в JSON
    json_data = json.dumps(payload)

    # Отправляем POST-запрос
    response = requests.post(api_url, data=json_data, headers=headers, timeout = 300, verify=False)

    # Проверяем статус ответа
    if response.status_code == 200:
        # Возвращаем ответ
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise ValueError(f"Ошибка запроса:{response.reason}")

In [13]:
def extract_json_from_response(response: str) -> dict:
    """
    Извлекает JSON-данные из текста ответа.

    Args:
        response (str): Текст ответа, который может содержать JSON в блоке ```json

    Returns:
        dict: Извлеченный словарь с данными или пустой словарь, если JSON не найден или невалиден.
    """
    pattern = r'```json\n(\{[\s\S]*?\})\n```'
    match = re.search(pattern, response, re.DOTALL)  # re.DOTALL для многострочного поиска

    if not match:
        return {}  # JSON не найден

    json_str = match.group(1)
    try:
        return json.loads(json_str)  # Парсим JSON в словарь
    except json.JSONDecodeError:
        return {}  # Невалидный JSON

In [14]:
def generate_prompt_IT_vacancies(title, text, vacancies):
    prefix = """
На основе изначального названия вакансии и её описания, сопоставь информацию с общим списком вакансий.
Обязательно выбери один из пунктов из списка вакансий.
Формат вывода:

```json
{
  "IT vacancy": "вакансия из списка вакансий"
}
```
Словарь не должен быть пустым! Словарь должен содержать ключ "IT vacancy".
"""

    prefix += f"\nНазвание вакансии: {title}\n"
    prefix += f"Описание вакансии: {text}\n"
    prefix += f"Список вакансий: {str(vacancies)}\n"
    return prefix

In [ ]:

from datetime import datetime


INPUT_DIR = Path("LS_source")
OUTPUT_DIR = Path("csv_tg_labeled_position")
OUTPUT_DIR.mkdir(exist_ok=True)

# функция для генерации промпта именно для столбца position
def generate_prompt_position(title, text, vacancies):
    prefix = """
На основе названия и описания вакансии, выбери одну профессию из списка.
Формат ответа:

```json
{
  "position": "вакансия из списка"
}
"""
    prefix += f"\nНазвание вакансии: {title}\n"
    prefix += f"Описание вакансии: {text}\n"
    prefix += f"Список вакансий: {str(vacancies)}\n"
    return prefix

def process_vacancies(input_file, output_file, partial_file):
    df = pd.read_csv(input_file)
    df = df.dropna(subset=["vacancy"]) # удаляем пустые строки по vacancy
    # Заполняем пустые даты текущей датой до секунд
    now_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    df["date"] = df["date"].fillna(now_str)

    df["uid"] = df["date"].astype(str) + "_" + df.index.astype(str)

    if os.path.exists(partial_file):
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial["uid"])
        print(f"Найден частичный файл: {partial_file} — уже обработано {len(processed_ids)} строк.")
    else:
        df_partial = pd.DataFrame(columns=df.columns.tolist() + ["position"])
        processed_ids = set()

    total = len(df)
    for idx, row in df.iterrows():
        uid = row["uid"]
        if uid in processed_ids:
            continue

        print(f"Обработка {idx+1}/{total} — {row['vacancy']}")
        prompt = generate_prompt_position(row["vacancy"], row["text"], vacancies)
        try:
            response = send_request_to_vllm(api_url, prompt, model_name)
            result = extract_json_from_response(response)
            position_value = result.get("position", "")
        except Exception as e:
            print(f"Ошибка при обработке строки {idx}: {e}")
            position_value = ""

        row_data = row.to_dict()
        row_data["position"] = position_value

        new_row_df = pd.DataFrame([row_data])
        new_row_df = new_row_df.reindex(columns=df_partial.columns)
        df_partial = pd.concat([df_partial, new_row_df], ignore_index=True)

        df_partial.to_csv(partial_file, index=False)

    df_partial.to_csv(output_file, index=False)
    print(f"Файл сохранен: {output_file}")

    if os.path.exists(partial_file):
        os.remove(partial_file)

In [ ]:
#=== Запуск обработки ===

csv_files = list(INPUT_DIR.glob("*.csv"))
for csv_path in csv_files:
    file_name = csv_path.stem
    output_file = OUTPUT_DIR / f"{file_name}_position.csv"
    partial_file = OUTPUT_DIR / f"{file_name}_partial.csv"
    # Пропускаем уже готовые файлы
    if output_file.exists():
        print(f"Файл уже обработан: {output_file.name}, пропуск.")
        continue

    print(f"\n=== Обработка файла: {csv_path.name} ===")
    process_vacancies(str(csv_path), str(output_file), str(partial_file))

In [ ]:
#  все поля из text с дополнительными условиями и промежуточным сохранением 

import requests
import json
import pandas as pd
import time
import os

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по разметке вакансий. 
1. Если в тексте есть описание вакансии, извлеки следующие поля:
vacancy, company, city, country, experience, employment, schedule, salary_from, salary_to, currency, skills, responsibilities, requirements, experience level.
2. Если зарплата в тексте не указана — верни salary_from и salary_to как null.
3. Если текст - это не описания вакансии, например объявление о поиске работы, верни null.
4. Заполни поле `experience` одним из следующих значений:

- "Нет опыта"
- "1–3 года"
- "3–6 лет"
- "Более 6 лет"

Если информация об опыте не указана, поставь null.

5. Заполни поле `schedule` одним из следующих значений:

- "Полный день" - если работа в офисе
- "Удаленная работа" - если работа удалённая
- "Гибкий график" - если гибрид офис / удаленка
- "Сменный график"
- "Вахтовый метод"

Если информация об опыте не указана, поставь null.

6.  Заполни поле `experience level` одним из следующих значений: 

- "Unknown",
- "Junior",
- "Trainee",
- "Middle",
- "Senior",
- "Lead"


Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст вакансии:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.2,
                    "max_tokens": 6000
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"]
            content = content.replace("```json", "").replace("```", "").strip()

            if content.lower() == "null":
                return None

            return json.loads(content)

        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса (попытка {attempt+1}):", e)
            time.sleep(2)
        except json.JSONDecodeError as e:
            print("Ошибка парсинга JSON:", e)
            print("Сырой ответ:", content)
            return None

    return None

def process_vacancies(input_file, output_file, partial_file='data/output_partial_tg_web.csv', id_column='Unnamed: 0'):
    df = pd.read_csv(input_file)

    # Восстановление прогресса
    if os.path.exists(partial_file):
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial[id_column])
        print(f"Найден частичный файл. Уже обработано: {len(processed_ids)} вакансий.")
    else:
        df_partial = pd.DataFrame()
        processed_ids = set()

    for i, row in df.iterrows():
        vacancy_id = row[id_column]
        if vacancy_id in processed_ids:
            continue

        print(f"Обработка вакансии {i+1}/{len(df)} (ID: {vacancy_id})...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        if result is None:
            row_result = {
                id_column: vacancy_id,
                "vacancy": "", "company": "", "city": "", "country": "",
                "experience": "", "employment": "", "schedule": "",
                "salary_from": None, "salary_to": None, "currency": "",
                "skills": [], "responsibilities": [], "requirements": [], "experience level": []
            }
        else:
            row_result = {
                id_column: vacancy_id,
                "vacancy": result.get("vacancy", ""),
                "company": result.get("company", ""),
                "city": result.get("city", ""),
                "country": result.get("country", ""),
                "experience": result.get("experience", ""),
                "employment": result.get("employment", ""),
                "schedule": result.get("schedule", ""),
                "salary_from": result.get("salary_from"),
                "salary_to": result.get("salary_to"),
                "currency": result.get("currency", ""),
                "skills": result.get("skills", []),
                "responsibilities": result.get("responsibilities", []),
                "requirements": result.get("requirements", []),
                "experience level": result.get("experience level", [])
            }

        # Сохраняем результат построчно
        df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2.5)  # таймаут между запросами

    # Финальное сохранение
    df_merged = pd.merge(df, df_partial, on=id_column, how="left")
    df_merged.to_csv(output_file, index=False)
    print(f"Готово! Финальный результат сохранён в {output_file}")

In [8]:
INPUT_CSV = "data/tg_web_to_mistral.csv"
OUTPUT_CSV = "data/tg_web_from_mistral.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Найден частичный файл. Уже обработано: 4372 вакансий.
Обработка вакансии 4373/11394 (ID: 5244)...
Ошибка парсинга JSON: Extra data: line 25 column 1 (char 574)
Сырой ответ: {
  "vacancy": "Front-end developer",
  "company": "Platinum",
  "city": null,
  "country": null,
  "experience": null,
  "employment": "Полная занятость",
  "schedule": "Удаленная работа",
  "salary_from": 2800,
  "salary_to": 3000,
  "currency": "$",
  "skills": ["react", "html", "css", "next.js", "web3"],
  "responsibilities": [
    "Создавать фронтенд приложения",
    "Уметь верстать под любые устройства",
    "Уметь работать как с REST так и WebSocket",
    "Опыт на node.js и работа с апи metamask"
  ],
  "requirements": null,
  "experience level": "Middle"
}



{
  "vacancy": "Back-end developer",
  "company": "Platinum",
  "city": null,
  "country": null,
  "experience": null,
  "employment": "Полная занятость",
  "schedule": "Удаленная работа",
  "salary_from": 2800,
  "salary_to": 3000,
  "currency": "$",
 

C:\Users\Андрей\AppData\Local\Temp\ipykernel_10176\2299690037.py:144: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)


Обработка вакансии 4374/11394 (ID: 5246)...
Обработка вакансии 4375/11394 (ID: 5248)...
Обработка вакансии 4376/11394 (ID: 5249)...
Обработка вакансии 4377/11394 (ID: 5250)...
Обработка вакансии 4378/11394 (ID: 5251)...
Обработка вакансии 4379/11394 (ID: 5252)...
Обработка вакансии 4380/11394 (ID: 5253)...
Обработка вакансии 4381/11394 (ID: 5254)...
Обработка вакансии 4382/11394 (ID: 5255)...
Обработка вакансии 4383/11394 (ID: 5256)...
Обработка вакансии 4384/11394 (ID: 5257)...
Обработка вакансии 4385/11394 (ID: 5258)...
Обработка вакансии 4386/11394 (ID: 5260)...
Обработка вакансии 4387/11394 (ID: 5261)...
Обработка вакансии 4388/11394 (ID: 5262)...
Обработка вакансии 4389/11394 (ID: 5268)...
Обработка вакансии 4390/11394 (ID: 5270)...
Обработка вакансии 4391/11394 (ID: 5271)...
Обработка вакансии 4392/11394 (ID: 5273)...
Обработка вакансии 4393/11394 (ID: 5275)...
Обработка вакансии 4394/11394 (ID: 5279)...
Обработка вакансии 4395/11394 (ID: 5280)...
Обработка вакансии 4396/11394 (I

In [ ]:
# Россия / Не Россия

import requests
import json
import pandas as pd
import time

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по анализу объявлений о IT-вакансиях ,
проанализируй текст и верни 'Россия' если по объявлению можно работать из России или 'Не Россия' только в противоположном случае.
Ответ запиши в поле country
Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.

Если информации нет — верни country как null.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст вакансии:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.2,
                    "max_tokens": 3000
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"]
            content = content.replace("```json", "").replace("```", "").strip()

            if content.lower() == "null":
                return None

            return json.loads(content)

        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса (попытка {attempt+1}):", e)
            time.sleep(2)
        except json.JSONDecodeError as e:
            print("Ошибка парсинга JSON:", e)
            print("Сырой ответ:", content)
            return None

    return None


# код с промежуточным сохранением 

def process_vacancies(input_file, output_file, partial_file='output_partial_miss_country.csv'):
    df = pd.read_csv(input_file)

    # Если уже есть частичный файл, восстановим прогресс  id_column='Unnamed: 0'
    try:
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial['Unnamed: 0'])
        print(f"Продолжаем с {len(processed_ids)} уже обработанных вакансий.")
    except FileNotFoundError:
        df_partial = pd.DataFrame()
        processed_ids = set()

    results = []

    for i, row in df.iterrows():
        if row['Unnamed: 0'] in processed_ids:
            continue

        print(f"Обработка вакансии {i+1}/{len(df)} ID={row['Unnamed: 0']}...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        row_result = row.to_dict()
        if result is None:
            row_result.update({
                "country": None
            })
        else:
            row_result.update({
                "country": result.get("country")
            })

        results.append(row_result)

        # Промежуточное сохранение после каждой вакансии
        df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2.5)

    # Финальное сохранение
    df_final = pd.read_csv(partial_file)
    df_final.to_csv(output_file, index=False)
    print(f"Готово! Полный результат сохранён в {output_file}")

In [10]:
INPUT_CSV = "texts_with_missing_country.csv"
OUTPUT_CSV = "texts_with_missing_country_mistral.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Обработка вакансии 1/3888 ID=0...
Обработка вакансии 2/3888 ID=8...
Обработка вакансии 3/3888 ID=9...
Обработка вакансии 4/3888 ID=10...
Обработка вакансии 5/3888 ID=14...
Обработка вакансии 6/3888 ID=16...
Обработка вакансии 7/3888 ID=21...
Обработка вакансии 8/3888 ID=40...
Обработка вакансии 9/3888 ID=41...
Обработка вакансии 10/3888 ID=46...
Обработка вакансии 11/3888 ID=58...
Обработка вакансии 12/3888 ID=59...
Обработка вакансии 13/3888 ID=72...
Обработка вакансии 14/3888 ID=75...
Обработка вакансии 15/3888 ID=77...
Обработка вакансии 16/3888 ID=79...
Обработка вакансии 17/3888 ID=90...
Обработка вакансии 18/3888 ID=96...
Обработка вакансии 19/3888 ID=97...
Обработка вакансии 20/3888 ID=98...
Обработка вакансии 21/3888 ID=99...
Обработка вакансии 22/3888 ID=100...
Обработка вакансии 23/3888 ID=102...
Обработка вакансии 24/3888 ID=103...
Обработка вакансии 25/3888 ID=104...
Обработка вакансии 26/3888 ID=125...
Обработка вакансии 27/3888 ID=132...
Обработка вакансии 28/3888 ID=137.

In [ ]:
# регион с защитой 

import requests
import json
import pandas as pd
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 20
REQUEST_DELAY = 4
INPUT_FILE = "city_y_cleaned_2.csv"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты географический помощник. По каждому названию города определи регион России в которой он находится.

Ответ строго в формате JSON:
{
  "results": [
    {"region": "Ростовская область"},
    ...
  ]
}

Если регион не удаётся распознать или страна не Россия — верни null.
"""

def prepare_prompt(batch):
    cities = "\n".join([f"{i+1}. {city}" for i, city in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Определи регион РФ для следующих городов:\n{cities}\n\nОтвет строго в JSON по формату выше."}
    ]

def query_mistral(prompt, max_tokens=500):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()

            if content.startswith("```json"):
                content = content[7:-3].strip()

            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")

            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def extract_country(input_file):
    print("🔄 Загрузка текущего состояния файла...")
    df = pd.read_csv(input_file)
    if 'region' not in df.columns:
        df['region'] = None

    # Целевые строки: city непустой
    mask = df['city'].notna() & (df['city'].astype(str).str.strip() != "") & (df['region'].isna())
    target_df = df[mask].copy()

    cities = target_df['city'].astype(str).tolist()
    index_list = target_df.index.tolist()

    print(f"🔍 Найдено {len(cities)} городов без региона. Начинаем обработку...")

    for i in tqdm(range(0, len(cities), BATCH_SIZE)):
        # Загрузка последней версии файла (на случай падения)
        df = pd.read_csv(input_file)

        batch_cities = cities[i:i + BATCH_SIZE]
        batch_indices = index_list[i:i + BATCH_SIZE]

        # Пропустить батч, если все регионы уже определены
        already_done = all(pd.notna(df.loc[idx, 'region']) for idx in batch_indices)
        if already_done:
            continue

        prompt = prepare_prompt(batch_cities)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch_cities)]):
                idx = batch_indices[j]
                if isinstance(item, dict):  # Проверяем, что item — словарь
                    df.at[idx, 'region'] = item.get('region', None)
                elif isinstance(item, str):  # Если это строка (например, "Ростовская область")
                    df.at[idx, 'region'] = item
                else:  # Если None или другой тип
                    df.at[idx, 'region'] = None
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        df.to_csv(input_file, index=False)

        if i + BATCH_SIZE < len(cities):
            time.sleep(REQUEST_DELAY + random.random())

    print(f"\n✅ Обработка завершена. Файл сохранён: {input_file}")
    return df

# Запуск
if __name__ == "__main__":
    extract_country(INPUT_FILE)

🔄 Загрузка текущего состояния файла...
🔍 Найдено 128 городов без региона. Начинаем обработку...


100%|██████████| 7/7 [00:37<00:00,  5.31s/it]


✅ Обработка завершена. Файл сохранён: city_y_cleaned_2.csv


In [ ]:
# Валюта 

import requests
import json
import pandas as pd
import time

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по анализу IT-вакансий. Определи валюту зарплаты на основе:
1. Указания в тексте (RUB, USD, EUR и т.д.)
2. Гражданства (РФ/РБ - RUB, США - USD и т.д.)
3. Компании (международные - USD/EUR, российские - RUB)
4. Формата работы (офис в стране - местная валюта)
5. Упомянутых законов (ТК РФ - RUB)

Примеры:
- "Гражданство РФ" → RUB
- "Зарплата 5000$" → USD
- "Компания из США" → USD
- "по ТК РФ" → RUB
- "удаленно из любой страны" → USD (международные обычно USD)

Верни ТОЛЬКО код валюты (RUB, USD, EUR) или null если не определишь.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст вакансии:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Формируем запрос к API
            payload = {
                "model": MODEL_NAME,
                "messages": prompt,
                "temperature": 0.3,  # Низкая температура для детерминированных ответов
                "max_tokens": 100,   # Уменьшил, так как ответ должен быть коротким
                "response_format": {"type": "text"}
            }
            
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json=payload,
                timeout=120
            )
            response.raise_for_status()
            
            # Извлекаем и очищаем ответ
            content = response.json()["choices"][0]["message"]["content"].strip()
            
            # Удаляем возможные кавычки и лишние символы
            content = content.replace('"', '').replace("'", "").strip()
            
            # Проверяем допустимые коды валют
            currency_codes = ['RUB', 'USD', 'EUR', 'GBP', 'CNY']  # Можно расширить список
            for code in currency_codes:
                if code.lower() in content.lower():
                    return {"currency": code}
            
            # Дополнительные проверки для российских вакансий
            if any(word in content.lower() for word in ['тк рф', 'россия', 'рф', 'руб']):
                return {"currency": "RUB"}
                
            return {"currency": None}

        except requests.exceptions.RequestException as e:
            print(f"Ошибка сети (попытка {attempt+1}):", str(e))
            time.sleep(2)
        except json.JSONDecodeError as e:
            print(f"Ошибка парсинга JSON (попытка {attempt+1}):", str(e))
            time.sleep(2)
        except KeyError as e:
            print(f"Ошибка структуры ответа (попытка {attempt+1}):", str(e))
            time.sleep(2)
    
    return {"currency": None}  # Все попытки исчерпаны


# код с промежуточным сохранением 

def process_vacancies(input_file, output_file, partial_file='output_partial_currency.csv'):
    df = pd.read_csv(input_file)

    # Если уже есть частичный файл, восстановим прогресс  id_column='Unnamed: 0'
    try:
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial['Unnamed: 0'])
        print(f"Продолжаем с {len(processed_ids)} уже обработанных вакансий.")
    except FileNotFoundError:
        df_partial = pd.DataFrame()
        processed_ids = set()

    results = []

    for i, row in df.iterrows():
        if row['Unnamed: 0'] in processed_ids:
            continue

        print(f"Обработка вакансии {i+1}/{len(df)} ID={row['Unnamed: 0']}...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        row_result = row.to_dict()
        if result is None:
            row_result.update({
                "currency": None
            })
        else:
            row_result.update({
                "currency": result.get("currency")
            })

        results.append(row_result)

        # Промежуточное сохранение после каждой вакансии
        df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2.5)

    # Финальное сохранение
    df_final = pd.read_csv(partial_file)
    df_final.to_csv(output_file, index=False)
    print(f"Готово! Полный результат сохранён в {output_file}")

In [23]:
INPUT_CSV = "check_currency_to_mistral.csv"
OUTPUT_CSV = "check_currency_from_mistral.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Обработка вакансии 1/491 ID=8...
Обработка вакансии 2/491 ID=9...
Обработка вакансии 3/491 ID=12...
Обработка вакансии 4/491 ID=13...
Обработка вакансии 5/491 ID=15...
Обработка вакансии 6/491 ID=17...
Обработка вакансии 7/491 ID=25...
Обработка вакансии 8/491 ID=34...
Обработка вакансии 9/491 ID=35...
Обработка вакансии 10/491 ID=54...
Обработка вакансии 11/491 ID=56...
Обработка вакансии 12/491 ID=61...
Обработка вакансии 13/491 ID=68...
Обработка вакансии 14/491 ID=80...
Обработка вакансии 15/491 ID=89...
Обработка вакансии 16/491 ID=99...
Обработка вакансии 17/491 ID=102...
Обработка вакансии 18/491 ID=109...
Обработка вакансии 19/491 ID=121...
Обработка вакансии 20/491 ID=128...
Обработка вакансии 21/491 ID=133...
Обработка вакансии 22/491 ID=157...
Обработка вакансии 23/491 ID=161...
Обработка вакансии 24/491 ID=166...
Обработка вакансии 25/491 ID=171...
Обработка вакансии 26/491 ID=196...
Обработка вакансии 27/491 ID=198...
Обработка вакансии 28/491 ID=199...
Обработка вакансии 

In [ ]:
# Резюме / Вакансия 

import requests
import json
import pandas as pd
import time

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по анализу IT-вакансий. Определи текст - объявление о вакансии или объявление о поиске работы (краткое резюме) на основе:
1. Хэштэги в тексте: #резюме, #resume — указывают на резюме; #вакансия, #vacancy, #job — указывают на вакансию
2. Формат подачи объявления: 
   - "Рассматриваю позицию...", "Ищу работу...", "Open to work" — резюме
   - "Ищем специалиста...", "Компания X ищет...", "Требуется..." — вакансия
3. Ключевые слова:
   - Для резюме: "мой опыт", "ищу работу", "готов рассмотреть"
   - Для вакансии: "требования", "обязанности", "условия работы"
4. Первое лицо (я, мне) обычно указывает на резюме, третье лицо (компания, мы) — на вакансию

Верни ТОЛЬКО тип сообщения: "вакансия", "резюме" или "null" если не определишь.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст объявления:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            payload = {
                "model": MODEL_NAME,
                "messages": prompt,
                "temperature": 0.3,
                "max_tokens": 15,
                "response_format": {"type": "text"}
            }
            
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json=payload,
                timeout=120
            )
            response.raise_for_status()
            
            content = response.json()["choices"][0]["message"]["content"].strip().lower()
            content = content.replace('"', '').replace("'", "").strip()
            
            if "вакансия" in content:
                return {"text_type": "вакансия"}
            elif "резюме" in content:
                return {"text_type": "резюме"}
            else:
                return {"text_type": None}

        except requests.exceptions.RequestException as e:
            print(f"Ошибка сети (попытка {attempt+1}):", str(e))
            time.sleep(2)
        except json.JSONDecodeError as e:
            print(f"Ошибка парсинга JSON (попытка {attempt+1}):", str(e))
            time.sleep(2)
        except KeyError as e:
            print(f"Ошибка структуры ответа (попытка {attempt+1}):", str(e))
            time.sleep(2)
    
    return {"text_type": None}


def process_vacancies(input_file, output_file, partial_file='output_partial_text_type.csv'):
    # Загружаем данные
    df = pd.read_csv(input_file)
    
    # Проверяем существование частичного файла
    try:
        df_partial = pd.read_csv(partial_file)
        processed_indices = set(df_partial.index)
        print(f"Продолжаем с {len(processed_indices)} уже обработанных записей.")
    except FileNotFoundError:
        df_partial = pd.DataFrame()
        processed_indices = set()

    results = []

    for idx, row in df.iterrows():
        if idx in processed_indices:
            continue

        print(f"Обработка записи {idx+1}/{len(df)}...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        row_result = row.to_dict()
        row_result["text_type"] = result.get("text_type")
        results.append(row_result)

        # Промежуточное сохранение
        temp_df = pd.DataFrame([row_result])
        df_partial = pd.concat([df_partial, temp_df], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2)

    # Финальное сохранение
    pd.DataFrame(results).to_csv(output_file, index=False)
    print(f"Готово! Полный результат сохранён в {output_file}")

In [3]:
INPUT_CSV = "data_final_stage5.csv"
OUTPUT_CSV = "final_stage5_mistral_text_type_checked.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Обработка записи 1/8633...
Обработка записи 2/8633...
Обработка записи 3/8633...
Обработка записи 4/8633...
Обработка записи 5/8633...
Обработка записи 6/8633...
Обработка записи 7/8633...
Обработка записи 8/8633...
Обработка записи 9/8633...
Обработка записи 10/8633...
Обработка записи 11/8633...
Обработка записи 12/8633...
Обработка записи 13/8633...
Обработка записи 14/8633...
Обработка записи 15/8633...
Обработка записи 16/8633...
Обработка записи 17/8633...
Обработка записи 18/8633...
Обработка записи 19/8633...
Обработка записи 20/8633...
Обработка записи 21/8633...
Обработка записи 22/8633...
Обработка записи 23/8633...
Обработка записи 24/8633...
Обработка записи 25/8633...
Обработка записи 26/8633...
Обработка записи 27/8633...
Обработка записи 28/8633...
Обработка записи 29/8633...
Обработка записи 30/8633...
Обработка записи 31/8633...
Обработка записи 32/8633...
Обработка записи 33/8633...
Обработка записи 34/8633...
Обработка записи 35/8633...
Обработка записи 36/8633...
О

In [ ]:
# it / not_it 

import requests
import json
import pandas as pd
import time

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по анализу IT-вакансий. Проанализируй текст объявление о вакансии и определи -  это IT профессия/вакансия или нет.
Если в тексте задачи - продавать услуги, руководить командой по продажам , заниматься охраной, кредитованием, страхованием, логистикой, закупкой и т.д. - это не IT  

Верни ТОЛЬКО тип вакансии: "it", "not_it" или "null" если не определишь.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст объявления:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            payload = {
                "model": MODEL_NAME,
                "messages": prompt,
                "temperature": 0.3,
                "max_tokens": 15,
                "response_format": {"type": "text"}
            }
            
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json=payload,
                timeout=120
            )
            response.raise_for_status()
            
            content = response.json()["choices"][0]["message"]["content"].strip().lower()
            content = content.replace('"', '').replace("'", "").strip()
            
            if "it" in content:
                return {"is_it": "it"}
            elif "not_it" in content:
                return {"is_it": "not_it"}
            else:
                return {"is_it": None}

        except requests.exceptions.RequestException as e:
            print(f"Ошибка сети (попытка {attempt+1}):", str(e))
            time.sleep(2)
        except json.JSONDecodeError as e:
            print(f"Ошибка парсинга JSON (попытка {attempt+1}):", str(e))
            time.sleep(2)
        except KeyError as e:
            print(f"Ошибка структуры ответа (попытка {attempt+1}):", str(e))
            time.sleep(2)
    
    return {"is_it": None}


def process_vacancies(input_file, output_file, partial_file='output_partial_it_type.csv'):
    # Загружаем данные
    df = pd.read_csv(input_file)
    
    # Проверяем существование частичного файла
    try:
        df_partial = pd.read_csv(partial_file)
        processed_indices = set(df_partial.index)
        print(f"Продолжаем с {len(processed_indices)} уже обработанных записей.")
    except FileNotFoundError:
        df_partial = pd.DataFrame()
        processed_indices = set()

    results = []

    for idx, row in df.iterrows():
        if idx in processed_indices:
            continue

        print(f"Обработка записи {idx+1}/{len(df)}...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        row_result = row.to_dict()
        row_result["is_it"] = result.get("is_it")
        results.append(row_result)

        # Промежуточное сохранение
        temp_df = pd.DataFrame([row_result])
        df_partial = pd.concat([df_partial, temp_df], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2)

    # Финальное сохранение
    pd.DataFrame(results).to_csv(output_file, index=False)
    print(f"Готово! Полный результат сохранён в {output_file}")

In [6]:
INPUT_CSV = "not_it_check.csv"
OUTPUT_CSV = "mistral_not_it_checked.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Обработка записи 1/99...
Обработка записи 2/99...
Обработка записи 3/99...
Обработка записи 4/99...
Обработка записи 5/99...
Обработка записи 6/99...
Обработка записи 7/99...
Обработка записи 8/99...
Обработка записи 9/99...
Обработка записи 10/99...
Обработка записи 11/99...
Обработка записи 12/99...
Обработка записи 13/99...
Обработка записи 14/99...
Обработка записи 15/99...
Обработка записи 16/99...
Обработка записи 17/99...
Обработка записи 18/99...
Обработка записи 19/99...
Обработка записи 20/99...
Обработка записи 21/99...
Обработка записи 22/99...
Обработка записи 23/99...
Обработка записи 24/99...
Обработка записи 25/99...
Обработка записи 26/99...
Обработка записи 27/99...
Обработка записи 28/99...
Обработка записи 29/99...
Обработка записи 30/99...
Обработка записи 31/99...
Обработка записи 32/99...
Обработка записи 33/99...
Обработка записи 34/99...
Обработка записи 35/99...
Обработка записи 36/99...
Обработка записи 37/99...
Обработка записи 38/99...
Обработка записи 39/9